In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pwd

/content


In [0]:
import os
os.chdir('/content/drive/My Drive/AFC_Project_data/')

# **Imports and Models**

In [0]:
import os
import argparse
from tqdm.notebook import tqdm
import numpy as np
from itertools import cycle

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
from torchvision.utils import make_grid
import torch.nn.functional as F

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
plt.switch_backend('agg')
from mpl_toolkits.mplot3d import Axes3D
from torch.utils.data import DataLoader
from sklearn import manifold

import random
from torchvision.utils import save_image

import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict

import numpy as np
from itertools import cycle
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader

from torch.distributions.normal import Normal

In [5]:
is_training = False
print(torch.cuda.is_available())

True


In [6]:
class Encoder(nn.Module):
		def __init__(self):
				super(Encoder, self).__init__()
				self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=2, bias=True)
				self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, bias=True)
				self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=2, bias=True)
				self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=2, bias=True)
				self.linear_mean_v = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_std_v = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_mean_a = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_std_a = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_mean_z = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_std_z = nn.Linear(in_features=128, out_features=32, bias=True)				
				self.features = nn.Linear(in_features=4608, out_features=512, bias=True)
				self.fc = nn.Linear(in_features=512, out_features=128)
				self.bn_1 = nn.BatchNorm2d(64)
				self.bn_2 = nn.BatchNorm2d(128)
				self.bn_3 = nn.BatchNorm2d(256)
				self.bn_4 = nn.BatchNorm2d(512)
				self.bn_5 = nn.BatchNorm1d(512)
				self.bn_6 = nn.BatchNorm1d(128)
				self.relu = nn.ReLU()
				self.elu = nn.ELU(0.1)
		
		def reparameterize(self, mu, logvar):
				if is_training:
					std = logvar.mul(0.5).exp_()
					eps = torch.randn_like(std)
					if(torch.cuda.is_available()):
						eps = eps.cuda()
					return eps.mul(std).add_(mu)
				else:
					return mu

		def forward(self, x):
			x = self.relu(self.bn_1(self.conv1(x)))
			x = self.relu(self.bn_2(self.conv2(x)))
			x = self.relu(self.bn_3(self.conv3(x)))
			x = self.relu(self.bn_4(self.conv4(x)))
			x = x.view(x.size(0), x.size(1)*x.size(2)*(x.size(3)))
			
			latent_feats = self.relu(self.bn_6(self.fc(self.relu(self.bn_5(self.features(x))))))
			mu_v = self.linear_mean_v(latent_feats)
			logvar_v = self.linear_std_v(latent_feats)
			mu_a = self.linear_mean_a(latent_feats)
			logvar_a = self.linear_std_a(latent_feats)
			mu_z = self.linear_mean_z(latent_feats)
			logvar_z = self.linear_std_z(latent_feats)	 	 
			z_v = self.reparameterize(mu_v, logvar_v)
			z_a = self.reparameterize(mu_a, logvar_a)
			z_z = self.reparameterize(mu_z, logvar_z)	 	 
			return z_v, mu_v, logvar_v, z_a, mu_a, logvar_a, z_z, mu_z, logvar_z

class Decoder(nn.Module):
		def __init__(self):
				super(Decoder, self).__init__()
				self.inv_fc = nn.Linear(in_features=128, out_features=512, bias=True)
				self.features = nn.Linear(in_features=512, out_features=4608, bias=True)
				self.reconstruct = nn.Linear(in_features=96, out_features=128, bias=True)
				self.deconv_1 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, stride=2, bias=True)
				self.deconv_2 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, stride=2, bias=True)
				self.deconv_3 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, bias=True)
				self.deconv_4 = nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=4, stride=2, bias=True)
				self.leaky_relu = nn.LeakyReLU(0.2)

				self.bn_1 = nn.BatchNorm1d(num_features=512)
				self.bn_2 = nn.BatchNorm1d(num_features=4608)
				self.bn_3 = nn.BatchNorm2d(num_features=256)
				self.bn_4 = nn.BatchNorm2d(num_features=128)
				self.bn_5 = nn.BatchNorm2d(num_features=64)
				self.bn_6 = nn.BatchNorm2d(num_features=3)

				self.tanh = nn.Tanh()
				self.relu =nn.ReLU()

		def forward(self, z_v, z_a, z_z):
			concat = torch.cat([z_v, z_a], dim=1)
			concat = torch.cat([concat, z_z], dim=1)
			latent = self.relu(self.bn_1(self.inv_fc(self.relu(self.reconstruct(concat)))))
			latent = self.relu(self.bn_2(self.features(latent)))
			x = latent.view(latent.size(0), 512, 3, 3)
			x = self.relu(self.bn_3(self.deconv_1(x)))
			x = self.relu(self.bn_4(self.deconv_2(x)))
			x = self.relu(self.bn_5(self.deconv_3(x)))
			x = self.relu(self.bn_6(self.deconv_4(x)))
			return x

enc = Encoder()
enc.cuda()
z_v,_,_, z_a, _, _, z_z, _, _ = enc.forward(torch.randn(10, 3, 64, 64).cuda())
print('Latents:', z_v.shape, z_a.shape, z_z.shape)

dec = Decoder()
dec.cuda()
i = dec.forward(z_v, z_a, z_z)
print('Recons:', i.shape)

Latents: torch.Size([10, 32]) torch.Size([10, 32]) torch.Size([10, 32])
Recons: torch.Size([10, 3, 64, 64])


In [7]:
class AFEW_Reg_Encoder(nn.Module):
		def __init__(self):
				super(AFEW_Reg_Encoder, self).__init__()
				self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=2, bias=True)
				self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, bias=True)
				self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=2, bias=True)
				self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=2, bias=True)
				self.linear_mean_v = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_std_v = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_mean_a = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_std_a = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_mean_z = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_std_z = nn.Linear(in_features=128, out_features=32, bias=True)	

				self.labels_v = nn.Linear(in_features=32, out_features=21, bias=True)
				self.labels_a = nn.Linear(in_features=32, out_features=21, bias=True)

				self.features = nn.Linear(in_features=4608, out_features=512, bias=True)
				self.fc = nn.Linear(in_features=512, out_features=128)
				self.bn_1 = nn.BatchNorm2d(64)
				self.bn_2 = nn.BatchNorm2d(128)
				self.bn_3 = nn.BatchNorm2d(256)
				self.bn_4 = nn.BatchNorm2d(512)
				self.bn_5 = nn.BatchNorm1d(512)
				self.bn_6 = nn.BatchNorm1d(128)
				self.relu = nn.ReLU()
				self.elu = nn.ELU(0.1)
		
		def reparameterize(self, mu, logvar):
				if is_training:
					std = logvar.mul(0.5).exp_()
					eps = torch.randn_like(std)
					if(torch.cuda.is_available()):
						eps = eps.cuda()
					return eps.mul(std).add_(mu)
				else:
					return mu

		def forward(self, x):
			x = self.relu(self.bn_1(self.conv1(x)))
			x = self.relu(self.bn_2(self.conv2(x)))
			x = self.relu(self.bn_3(self.conv3(x)))
			x = self.relu(self.bn_4(self.conv4(x)))
			x = x.view(x.size(0), x.size(1)*x.size(2)*(x.size(3)))
			
			latent_feats = self.relu(self.bn_6(self.fc(self.relu(self.bn_5(self.features(x))))))
			mu_v = self.linear_mean_v(latent_feats)
			logvar_v = self.linear_std_v(latent_feats)
			mu_a = self.linear_mean_a(latent_feats)
			logvar_a = self.linear_std_a(latent_feats)
			mu_z = self.linear_mean_z(latent_feats)
			logvar_z = self.linear_std_z(latent_feats)	 	 
			z_v = self.reparameterize(mu_v, logvar_v)
			z_a = self.reparameterize(mu_a, logvar_a)
			z_z = self.reparameterize(mu_z, logvar_z)	 
			
			label_v = self.labels_v(z_v)
			label_a = self.labels_a(z_a)

			return z_v, mu_v, logvar_v, z_a, mu_a, logvar_a, z_z, mu_z, logvar_z, label_v, label_a

class AFEW_Reg_Decoder(nn.Module):
		def __init__(self):
				super(AFEW_Reg_Decoder, self).__init__()
				self.inv_fc = nn.Linear(in_features=128, out_features=512, bias=True)
				self.features = nn.Linear(in_features=512, out_features=4608, bias=True)
				self.reconstruct = nn.Linear(in_features=96, out_features=128, bias=True)
				self.deconv_1 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, stride=2, bias=True)
				self.deconv_2 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, stride=2, bias=True)
				self.deconv_3 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, bias=True)
				self.deconv_4 = nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=4, stride=2, bias=True)
				self.leaky_relu = nn.LeakyReLU(0.2)

				self.bn_1 = nn.BatchNorm1d(num_features=512)
				self.bn_2 = nn.BatchNorm1d(num_features=4608)
				self.bn_3 = nn.BatchNorm2d(num_features=256)
				self.bn_4 = nn.BatchNorm2d(num_features=128)
				self.bn_5 = nn.BatchNorm2d(num_features=64)
				self.bn_6 = nn.BatchNorm2d(num_features=3)

				self.tanh = nn.Tanh()
				self.relu =nn.ReLU()

		def forward(self, z_v, z_a, z_z):
			concat = torch.cat([z_v, z_a], dim=1)
			concat = torch.cat([concat, z_z], dim=1)
			latent = self.relu(self.bn_1(self.inv_fc(self.relu(self.reconstruct(concat)))))
			latent = self.relu(self.bn_2(self.features(latent)))
			x = latent.view(latent.size(0), 512, 3, 3)
			x = self.relu(self.bn_3(self.deconv_1(x)))
			x = self.relu(self.bn_4(self.deconv_2(x)))
			x = self.relu(self.bn_5(self.deconv_3(x)))
			x = self.relu(self.bn_6(self.deconv_4(x)))
			return x

enc = AFEW_Reg_Encoder()
enc.cuda()
z_v,_,_, z_a, _, _, z_z, _, _, l_v, l_a = enc.forward(torch.randn(10, 3, 64, 64).cuda())
print('Latents:', z_v.shape, z_a.shape, z_z.shape, l_v.shape, l_a.shape)

dec = AFEW_Reg_Decoder()
dec.cuda()
i = dec.forward(z_v, z_a, z_z)
print('Recons:', i.shape)

Latents: torch.Size([10, 32]) torch.Size([10, 32]) torch.Size([10, 32]) torch.Size([10, 21]) torch.Size([10, 21])
Recons: torch.Size([10, 3, 64, 64])


In [8]:
class IMFDB_Reg_Discrete_Encoder(nn.Module):
		def __init__(self):
				super(IMFDB_Reg_Discrete_Encoder, self).__init__()
				self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=2, bias=True)
				self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, bias=True)
				self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=2, bias=True)
				self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=2, bias=True)
				self.linear_mean_v = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_std_v = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_mean_a = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_std_a = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_mean_z = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_std_z = nn.Linear(in_features=128, out_features=32, bias=True)	

				self.labels_v = nn.Linear(in_features=32, out_features=19, bias=True)
				self.labels_a = nn.Linear(in_features=32, out_features=19, bias=True)

				self.features = nn.Linear(in_features=4608, out_features=512, bias=True)
				self.fc = nn.Linear(in_features=512, out_features=128)
				self.bn_1 = nn.BatchNorm2d(64)
				self.bn_2 = nn.BatchNorm2d(128)
				self.bn_3 = nn.BatchNorm2d(256)
				self.bn_4 = nn.BatchNorm2d(512)
				self.bn_5 = nn.BatchNorm1d(512)
				self.bn_6 = nn.BatchNorm1d(128)
				self.relu = nn.ReLU()
				self.elu = nn.ELU(0.1)
		
		def reparameterize(self, mu, logvar):
				if is_training:
					std = logvar.mul(0.5).exp_()
					eps = torch.randn_like(std)
					if(torch.cuda.is_available()):
						eps = eps.cuda()
					return eps.mul(std).add_(mu)
				else:
					return mu

		def forward(self, x):
			x = self.relu(self.bn_1(self.conv1(x)))
			x = self.relu(self.bn_2(self.conv2(x)))
			x = self.relu(self.bn_3(self.conv3(x)))
			x = self.relu(self.bn_4(self.conv4(x)))
			x = x.view(x.size(0), x.size(1)*x.size(2)*(x.size(3)))
			
			latent_feats = self.relu(self.bn_6(self.fc(self.relu(self.bn_5(self.features(x))))))
			mu_v = self.linear_mean_v(latent_feats)
			logvar_v = self.linear_std_v(latent_feats)
			mu_a = self.linear_mean_a(latent_feats)
			logvar_a = self.linear_std_a(latent_feats)
			mu_z = self.linear_mean_z(latent_feats)
			logvar_z = self.linear_std_z(latent_feats)	 	 
			z_v = self.reparameterize(mu_v, logvar_v)
			z_a = self.reparameterize(mu_a, logvar_a)
			z_z = self.reparameterize(mu_z, logvar_z)	 
			
			label_v = self.labels_v(z_v)
			label_a = self.labels_a(z_a)

			return z_v, mu_v, logvar_v, z_a, mu_a, logvar_a, z_z, mu_z, logvar_z, label_v, label_a

class IMFDB_Reg_Discrete_Decoder(nn.Module):
		def __init__(self):
				super(IMFDB_Reg_Discrete_Decoder, self).__init__()
				self.inv_fc = nn.Linear(in_features=128, out_features=512, bias=True)
				self.features = nn.Linear(in_features=512, out_features=4608, bias=True)
				self.reconstruct = nn.Linear(in_features=96, out_features=128, bias=True)
				self.deconv_1 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, stride=2, bias=True)
				self.deconv_2 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, stride=2, bias=True)
				self.deconv_3 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, bias=True)
				self.deconv_4 = nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=4, stride=2, bias=True)
				self.leaky_relu = nn.LeakyReLU(0.2)

				self.bn_1 = nn.BatchNorm1d(num_features=512)
				self.bn_2 = nn.BatchNorm1d(num_features=4608)
				self.bn_3 = nn.BatchNorm2d(num_features=256)
				self.bn_4 = nn.BatchNorm2d(num_features=128)
				self.bn_5 = nn.BatchNorm2d(num_features=64)
				self.bn_6 = nn.BatchNorm2d(num_features=3)

				self.tanh = nn.Tanh()
				self.relu =nn.ReLU()

		def forward(self, z_v, z_a, z_z):
			concat = torch.cat([z_v, z_a], dim=1)
			concat = torch.cat([concat, z_z], dim=1)
			latent = self.relu(self.bn_1(self.inv_fc(self.relu(self.reconstruct(concat)))))
			latent = self.relu(self.bn_2(self.features(latent)))
			x = latent.view(latent.size(0), 512, 3, 3)
			x = self.relu(self.bn_3(self.deconv_1(x)))
			x = self.relu(self.bn_4(self.deconv_2(x)))
			x = self.relu(self.bn_5(self.deconv_3(x)))
			x = self.relu(self.bn_6(self.deconv_4(x)))
			return x

enc = IMFDB_Reg_Discrete_Encoder()
enc.cuda()
z_v,_,_, z_a, _, _, z_z, _, _, l_v, l_a = enc.forward(torch.randn(10, 3, 64, 64).cuda())
print('Latents:', z_v.shape, z_a.shape, z_z.shape, l_v.shape, l_a.shape)

dec = IMFDB_Reg_Discrete_Decoder()
dec.cuda()
i = dec.forward(z_v, z_a, z_z)
print('Recons:', i.shape)

Latents: torch.Size([10, 32]) torch.Size([10, 32]) torch.Size([10, 32]) torch.Size([10, 19]) torch.Size([10, 19])
Recons: torch.Size([10, 3, 64, 64])


In [9]:
class IMFDB_Reg_Encoder(nn.Module):
		def __init__(self):
				super(IMFDB_Reg_Encoder, self).__init__()
				self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=2, bias=True)
				self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=2, bias=True)
				self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=2, bias=True)
				self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=2, bias=True)
				self.linear_mean_v = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_std_v = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_mean_a = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_std_a = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_mean_z = nn.Linear(in_features=128, out_features=32, bias=True)
				self.linear_std_z = nn.Linear(in_features=128, out_features=32, bias=True)	

				self.features = nn.Linear(in_features=4608, out_features=512, bias=True)
				self.fc = nn.Linear(in_features=512, out_features=128)
				self.bn_1 = nn.BatchNorm2d(64)
				self.bn_2 = nn.BatchNorm2d(128)
				self.bn_3 = nn.BatchNorm2d(256)
				self.bn_4 = nn.BatchNorm2d(512)
				self.bn_5 = nn.BatchNorm1d(512)
				self.bn_6 = nn.BatchNorm1d(128)
				self.relu = nn.ReLU()
				self.elu = nn.ELU(0.1)
		
		def reparameterize(self, mu, logvar):
				if is_training:
					std = logvar.mul(0.5).exp_()
					eps = torch.randn_like(std)
					if(torch.cuda.is_available()):
						eps = eps.cuda()
					return eps.mul(std).add_(mu)
				else:
					return mu

		def forward(self, x):
			x = self.relu(self.bn_1(self.conv1(x)))
			x = self.relu(self.bn_2(self.conv2(x)))
			x = self.relu(self.bn_3(self.conv3(x)))
			x = self.relu(self.bn_4(self.conv4(x)))
			x = x.view(x.size(0), x.size(1)*x.size(2)*(x.size(3)))
			
			latent_feats = self.relu(self.bn_6(self.fc(self.relu(self.bn_5(self.features(x))))))
			mu_v = self.linear_mean_v(latent_feats)
			logvar_v = self.linear_std_v(latent_feats)
			mu_a = self.linear_mean_a(latent_feats)
			logvar_a = self.linear_std_a(latent_feats)
			mu_z = self.linear_mean_z(latent_feats)
			logvar_z = self.linear_std_z(latent_feats)	 	 
			z_v = self.reparameterize(mu_v, logvar_v)
			z_a = self.reparameterize(mu_a, logvar_a)
			z_z = self.reparameterize(mu_z, logvar_z)	 

			return z_v, mu_v, logvar_v, z_a, mu_a, logvar_a, z_z, mu_z, logvar_z

class IMFDB_Reg_Decoder(nn.Module):
		def __init__(self):
				super(IMFDB_Reg_Decoder, self).__init__()
				self.inv_fc = nn.Linear(in_features=128, out_features=512, bias=True)
				self.features = nn.Linear(in_features=512, out_features=4608, bias=True)
				self.reconstruct = nn.Linear(in_features=96, out_features=128, bias=True)
				self.deconv_1 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, stride=2, bias=True)
				self.deconv_2 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, stride=2, bias=True)
				self.deconv_3 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, bias=True)
				self.deconv_4 = nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=4, stride=2, bias=True)
				self.leaky_relu = nn.LeakyReLU(0.2)

				self.bn_1 = nn.BatchNorm1d(num_features=512)
				self.bn_2 = nn.BatchNorm1d(num_features=4608)
				self.bn_3 = nn.BatchNorm2d(num_features=256)
				self.bn_4 = nn.BatchNorm2d(num_features=128)
				self.bn_5 = nn.BatchNorm2d(num_features=64)
				self.bn_6 = nn.BatchNorm2d(num_features=3)

				self.tanh = nn.Tanh()
				self.relu =nn.ReLU()

		def forward(self, z_v, z_a, z_z):
			concat = torch.cat([z_v, z_a], dim=1)
			concat = torch.cat([concat, z_z], dim=1)
			latent = self.relu(self.bn_1(self.inv_fc(self.relu(self.reconstruct(concat)))))
			latent = self.relu(self.bn_2(self.features(latent)))
			x = latent.view(latent.size(0), 512, 3, 3)
			x = self.relu(self.bn_3(self.deconv_1(x)))
			x = self.relu(self.bn_4(self.deconv_2(x)))
			x = self.relu(self.bn_5(self.deconv_3(x)))
			x = self.relu(self.bn_6(self.deconv_4(x)))
			return x

enc = IMFDB_Reg_Encoder()
enc.cuda()
z_v,_,_, z_a, _, _, z_z, _, _ = enc.forward(torch.randn(10, 3, 64, 64).cuda())
print('Latents:', z_v.shape, z_a.shape, z_z.shape)

dec = IMFDB_Reg_Decoder()
dec.cuda()
i = dec.forward(z_v, z_a, z_z)
print('Recons:', i.shape)

Latents: torch.Size([10, 32]) torch.Size([10, 32]) torch.Size([10, 32])
Recons: torch.Size([10, 3, 64, 64])


In [0]:
def mse_loss(input1, target):
	return torch.sum((input1 - target).pow(2)) / input1.data.nelement()
 
def mae_loss(input1, target):
	return abs(torch.sum((input1 - target).pow(1)) / input1.data.nelement())

def weights_init(layer):
	if isinstance(layer, nn.Conv2d):
		layer.weight.data.normal_(0.0, 0.05)
		layer.bias.data.zero_()
	elif isinstance(layer, nn.BatchNorm2d):
		layer.weight.data.normal_(1.0, 0.02)
		layer.bias.data.zero_()
	elif isinstance(layer, nn.Linear):
		layer.weight.data.normal_(0.0, 0.05)
		layer.bias.data.zero_()


def imshow_grid(images, shape=[2, 8], name='default', save=False):
	"""Plot images in a grid of a given shape."""
	fig = plt.figure(1)
	grid = ImageGrid(fig, 111, nrows_ncols=shape, axes_pad=0.05)

	size = shape[0] * shape[1]
	for i in range(size):
		grid[i].axis('off')
		grid[i].imshow(images[i])  # The AxesGrid object work as a list of axes.

	if save:
		plt.savefig('reconstructed_images/' + str(name) + '.png')
		plt.clf()
	else:
		plt.show()

# **IMFDB Continuous VA analysis**


In [10]:
os.chdir('./IMFDB_data/processed_images')
!ls

AamairKhan	 Innocent	   Mamootty	       SharmilaTagore
Aarthi		 Jagadeesh	   Mohanlal	       SharukhKhan
AkshayKumar	 JagapathiBabu	   M.S.Narayana        Shashikumar
Ali		 Jagathi	   Mukesh	       ShilpaShetty
allimages	 Jamuna		   Nagarjuna	       Shivaram
Ambresh		 JayaBhaduri	   NedumudiVenu        Shobana
AmitabhBachchan  Jayaprakash	   NTR		       Siddique
AmrishPuri	 Jayram		   PareshRaval	       Simran
AnilKapoor	 JosePrakash	   Pavithralokesh      SoumithraChatterjee
Annapoorna	 KajalAgarwal	   PrakashRaj	       Soundarya
ANR		 Kajol		   Prema	       Srihari
AnupamKher	 KareenaKapoor	   PremaNazir	       SureshGopi
Ashwath		 KarunaBenerjee    RajeshKhanna        Suryakantham
Avinash		 KatrinaKaif	   RakheeGulzar        SVR
BabuMohan	 KavyaMadhavan	   RamanaReddy	       TanikellaBharani
Balakrishna	 KotaSrinivasarao  Ramaprabha	       Tenniskrishna
Bhanupriya	 K.Viswanath	   RameshArvind        Thilakan
Bharathi	 Lakshmidevi	   Rameshbhatt	       Trisha
BomanIrani	 Leelavathi	 

In [13]:
  import json

  json_file = '/content/drive/My Drive/AFC_Project_data/IMFDB_data/IMFDB_va.json'

  with open(json_file) as f:
    va_dictionary = json.load(f)
  print(va_dictionary) 

Output hidden; open in https://colab.research.google.com to view.

In [14]:
import json

json_file = '/content/drive/My Drive/AFC_Project_data/IMFDB_data/IMFDB_labels.json'

with open(json_file) as f:
  label_dictionary = json.load(f)
print(label_dictionary) 

{'Gundammakatha_ANR_100.jpg': 'NEUTRAL', 'Gundammakatha_ANR_101.jpg': 'NEUTRAL', 'Gundammakatha_ANR_102.jpg': 'NEUTRAL', 'Gundammakatha_ANR_103.jpg': 'HAPPINESS', 'Gundammakatha_ANR_104.jpg': 'NEUTRAL', 'Gundammakatha_ANR_105.jpg': 'NEUTRAL', 'Gundammakatha_ANR_106.jpg': 'NEUTRAL', 'Gundammakatha_ANR_107.jpg': 'HAPPINESS', 'Gundammakatha_ANR_11.jpg': 'NEUTRAL', 'Gundammakatha_ANR_111.jpg': 'NEUTRAL', 'Gundammakatha_ANR_113.jpg': 'NEUTRAL', 'Gundammakatha_ANR_117.jpg': 'SURPRISE', 'Gundammakatha_ANR_118.jpg': 'NEUTRAL', 'Gundammakatha_ANR_12.jpg': 'NEUTRAL', 'Gundammakatha_ANR_120.jpg': 'SURPRISE', 'Gundammakatha_ANR_121.jpg': 'NEUTRAL', 'Gundammakatha_ANR_122.jpg': 'NEUTRAL', 'Gundammakatha_ANR_123.jpg': 'SADNESS', 'Gundammakatha_ANR_125.jpg': 'SURPRISE', 'Gundammakatha_ANR_126.jpg': 'SURPRISE', 'Gundammakatha_ANR_127.jpg': 'HAPPINESS', 'Gundammakatha_ANR_128.jpg': 'HAPPINESS', 'Gundammakatha_ANR_129.jpg': 'SURPRISE', 'Gundammakatha_ANR_130.jpg': 'HAPPINESS', 'Gundammakatha_ANR_131.jpg

In [0]:
import matplotlib.pyplot as plt

labels_dist = list(label_dictionary.values())

keys = list(np.unique(labels_dist, return_counts=True)[0])
vals = list(np.unique(labels_dist, return_counts=True)[1])

v = []
for val in keys:
  v.append(val[0:2])

fig = plt.figure()
plt.bar(v, vals)
plt.title('Emotion classes distribution in IMFDB')
plt.savefig('/content/drive/My Drive/AFC_Project_data/IMFDB_data/labels_distribution.png')

In [0]:
emotion_mapping = ['ANGER', 'DISGUST', 'FEAR', 'HAPPINESS', 'NEUTRAL', 'SADNESS', 'SURPRISE']

class ImageFolderWithPathsIMFDB(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPathsIMFDB, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        # folder = path[-13:-10]
        fil = path[path.rfind('/')+1:]
        emotion = (emotion_mapping.index(label_dictionary[fil]),)
        tuple_with_path = (original_tuple+(va_dictionary[fil],)+emotion)
        return tuple_with_path

## **Quantitative Comparison**

In [0]:
encoder = Encoder()
encoder.apply(weights_init)
encoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/IMFDB_data/encoder'))

decoder = Decoder()
decoder.apply(weights_init)
decoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/IMFDB_data/decoder'))

batch_size = 2

print('Loading Dataset...')
data_dir = '/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/'
transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])
dset = ImageFolderWithPathsIMFDB(data_dir, transform=transform)
print(dset.imgs[:10])
data_object = torch.utils.data.DataLoader(dset, batch_size=batch_size, shuffle=True, drop_last=True)

X1 = torch.zeros(batch_size, 3, 64, 64)

if torch.cuda.is_available():
  encoder.cuda()
  decoder.cuda()
  X1 = X1.cuda()

s_metric = 0
s_v_m = 0
s_a_m = 0

a_metric = 0
a_v_m = 0
a_a_m = 0

all_z_v = []
all_z_a = []

all_v_labels = []
all_a_labels = []
all_emotion_labels = []

with torch.no_grad():
  for iteration, (data_, _, labels, emotions) in tqdm(enumerate(data_object)):
      image_batch_1 = data_
      X1.copy_(image_batch_1)
      z_v, mu_v, logvar_v, z_a, mu_a, logvar_a, z_z, mu_z, logvar_z = encoder(X1)

      all_z_v = all_z_v + list(z_v.detach().cpu().numpy())
      all_z_a = all_z_a + list(z_a.detach().cpu().numpy())

      all_v_labels = all_v_labels + list(labels[0].detach().cpu().numpy())
      all_a_labels = all_a_labels + list(labels[1].detach().cpu().numpy())
      all_emotion_labels = all_emotion_labels + list(emotions.detach().cpu().numpy())

      sup_loss = mse_loss(z_v, labels[0].repeat(32, 1).t().cuda()) + mse_loss(z_a, labels[1].repeat(32, 1).t().cuda())
      v_l = mse_loss(z_v, labels[0].repeat(32, 1).t().cuda())
      a_l = mse_loss(z_a, labels[1].repeat(32, 1).t().cuda())

      sup_loss_a = mae_loss(z_v, labels[0].repeat(32, 1).t().cuda()) + mae_loss(z_a, labels[1].repeat(32, 1).t().cuda())
      v_l_a = mae_loss(z_v, labels[0].repeat(32, 1).t().cuda())
      a_l_a = mae_loss(z_a, labels[1].repeat(32, 1).t().cuda())

      s_metric += sup_loss.item()
      s_v_m += v_l.item()
      s_a_m += a_l.item()

      a_metric += sup_loss_a.item()
      a_v_m += v_l_a.item()
      a_a_m += a_l_a.item()

      if(iteration==1000):
          break

print(s_metric/iteration*batch_size, s_v_m/iteration*batch_size, s_a_m/iteration*batch_size)
print(a_metric/iteration*batch_size, a_v_m/iteration*batch_size, a_a_m/iteration*batch_size)

import pickle

pickle.dump(all_z_v, open('vanilla_vae_z_v.pickle', 'wb'))
pickle.dump(all_z_a, open('vanilla_vae_z_a.pickle', 'wb'))
pickle.dump(all_v_labels, open('vanilla_vae_v_labels.pickle', 'wb'))
pickle.dump(all_a_labels, open('vanilla_vae_a_labels.pickle', 'wb'))
pickle.dump(all_emotion_labels, open('vanilla_vae_emotion_labels.pickle', 'wb'))

Loading Dataset...
[('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_1.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_10.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_11.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_12.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_13.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_14.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_15.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_16.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_17.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_ima

13.267810590557783 7.306263866858637 5.961546723699144
2.219951707496022 1.1615580169149824 1.0583936905810354


In [0]:
encoder = IMFDB_Reg_Encoder()
encoder.apply(weights_init)
encoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/IMFDB_data/reg_encoder'))

decoder = IMFDB_Reg_Decoder()
decoder.apply(weights_init)
decoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/IMFDB_data/reg_decoder'))

batch_size = 2

print('Loading Dataset...')
data_dir = '/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/'
transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])
dset = ImageFolderWithPathsIMFDB(data_dir, transform=transform)
print(dset.imgs[:10])
data_object = torch.utils.data.DataLoader(dset, batch_size=batch_size, shuffle=True, drop_last=True)

X1 = torch.zeros(batch_size, 3, 64, 64)

if torch.cuda.is_available():
  encoder.cuda()
  decoder.cuda()
  X1 = X1.cuda()

s_metric = 0
s_v_m = 0
s_a_m = 0

a_metric = 0
a_v_m = 0
a_a_m = 0

all_z_v = []
all_z_a = []

all_v_labels = []
all_a_labels = []
all_emotion_labels = []

with torch.no_grad():
  for iteration, (data_, _, labels, emotions) in tqdm(enumerate(data_object)):
      image_batch_1 = data_
      X1.copy_(image_batch_1)
      z_v, mu_v, logvar_v, z_a, mu_a, logvar_a, z_z, mu_z, logvar_z = encoder(X1)

      all_z_v = all_z_v + list(z_v.detach().cpu().numpy())
      all_z_a = all_z_a + list(z_a.detach().cpu().numpy())

      all_v_labels = all_v_labels + list(labels[0].detach().cpu().numpy())
      all_a_labels = all_a_labels + list(labels[1].detach().cpu().numpy())
      all_emotion_labels = all_emotion_labels + list(emotions.detach().cpu().numpy())

      sup_loss = mse_loss(z_v, labels[0].repeat(32, 1).t().cuda()) + mse_loss(z_a, labels[1].repeat(32, 1).t().cuda())
      v_l = mse_loss(z_v, labels[0].repeat(32, 1).t().cuda())
      a_l = mse_loss(z_a, labels[1].repeat(32, 1).t().cuda())

      sup_loss_a = mae_loss(z_v, labels[0].repeat(32, 1).t().cuda()) + mae_loss(z_a, labels[1].repeat(32, 1).t().cuda())
      v_l_a = mae_loss(z_v, labels[0].repeat(32, 1).t().cuda())
      a_l_a = mae_loss(z_a, labels[1].repeat(32, 1).t().cuda())

      s_metric += sup_loss.item()
      s_v_m += v_l.item()
      s_a_m += a_l.item()

      a_metric += sup_loss_a.item()
      a_v_m += v_l_a.item()
      a_a_m += a_l_a.item()

      if(iteration==1000):
          break

print(s_metric/iteration*batch_size, s_v_m/iteration*batch_size, s_a_m/iteration*batch_size)
print(a_metric/iteration*batch_size, a_v_m/iteration*batch_size, a_a_m/iteration*batch_size)

import pickle

pickle.dump(all_z_v, open('reg_vae_z_v.pickle', 'wb'))
pickle.dump(all_z_a, open('reg_vae_z_a.pickle', 'wb'))
pickle.dump(all_v_labels, open('reg_vae_v_labels.pickle', 'wb'))
pickle.dump(all_a_labels, open('reg_vae_a_labels.pickle', 'wb'))
pickle.dump(all_emotion_labels, open('reg_vae_emotion_labels.pickle', 'wb'))

Loading Dataset...
[('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_1.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_10.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_11.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_12.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_13.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_14.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_15.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_16.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_17.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_ima

0.8308415883684955 0.5739840976891804 0.25685749067931524
0.9505304515266805 0.5827188158251813 0.3678116357014997


In [0]:
import pickle

data_dir = '/content/drive/My Drive/AFC_Project_data/IMFDB_data/'

reg_z_v = pickle.load(open(data_dir+'reg_vae_z_v.pickle', 'rb'))
reg_z_a = pickle.load(open(data_dir+'reg_vae_z_a.pickle', 'rb'))
reg_v_labels = pickle.load(open(data_dir+'reg_vae_v_labels.pickle', 'rb'))
reg_a_labels = pickle.load(open(data_dir+'reg_vae_a_labels.pickle', 'rb'))
reg_emotion_labels = pickle.load(open(data_dir+'reg_vae_emotion_labels.pickle', 'rb'))

vanilla_z_v = pickle.load(open(data_dir+'vanilla_vae_z_v.pickle', 'rb'))
vanilla_z_a = pickle.load(open(data_dir+'vanilla_vae_z_a.pickle', 'rb'))
vanilla_v_labels = pickle.load(open(data_dir+'vanilla_vae_v_labels.pickle', 'rb'))
vanilla_a_labels = pickle.load(open(data_dir+'vanilla_vae_a_labels.pickle', 'rb'))
vanilla_emotion_labels = pickle.load(open(data_dir+'vanilla_vae_emotion_labels.pickle', 'rb'))

print(len(reg_z_v), len(reg_z_a), len(reg_v_labels), len(reg_a_labels), len(reg_emotion_labels))
print(len(vanilla_z_v), len(vanilla_z_a), len(vanilla_v_labels), len(vanilla_a_labels), len(vanilla_emotion_labels))

2002 2002 2002 2002 2002
2002 2002 2002 2002 2002


### **Discrete Emotion Classification**

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier

preprocessing.scale(vanilla_z_v)
preprocessing.scale(vanilla_z_a)
preprocessing.scale(reg_z_v)
preprocessing.scale(reg_z_a)

X_train, X_test, y_train, y_test = train_test_split(vanilla_z_v, vanilla_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla V MLP: '+str(accuracy_score(y_test, preds)))

clf = clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla V SVM: '+str(accuracy_score(y_test, preds)))

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# preds = clf.predict(X_test)
# print('Vanilla V LR: '+str(accuracy_score(y_test, preds)))

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla V DT: '+str(accuracy_score(y_test, preds)))

X_train, X_test, y_train, y_test = train_test_split(vanilla_z_a, vanilla_emotion_labels, test_size=0.1, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla A MLP: '+str(accuracy_score(y_test, preds)))

clf = clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla A SVM: '+str(accuracy_score(y_test, preds)))

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# preds = clf.predict(X_test)
# print('Vanilla A LR: '+str(accuracy_score(y_test, preds)))

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla A DT: '+str(accuracy_score(y_test, preds)))

X_train, X_test, y_train, y_test = train_test_split(reg_z_v, reg_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V MLP: '+str(accuracy_score(y_test, preds)))

clf = clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V SVM: '+str(accuracy_score(y_test, preds)))

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# preds = clf.predict(X_test)
# print('Reg V LR: '+str(accuracy_score(y_test, preds)))

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V DT: '+str(accuracy_score(y_test, preds)))

X_train, X_test, y_train, y_test = train_test_split(reg_z_a, vanilla_emotion_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A MLP: '+str(accuracy_score(y_test, preds)))

clf = clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A SVM: '+str(accuracy_score(y_test, preds)))

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# preds = clf.predict(X_test)
# print('Reg A LR: '+str(accuracy_score(y_test, preds)))

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A DT: '+str(accuracy_score(y_test, preds)))

Vanilla V MLP: 0.3144758735440932
Vanilla V SVM: 0.36772046589018303
Vanilla V DT: 0.2079866888519135
Vanilla A MLP: 0.2935323383084577
Vanilla A SVM: 0.32338308457711445
Vanilla A DT: 0.18407960199004975
Reg V MLP: 0.3427620632279534
Reg V SVM: 0.33777038269550747
Reg V DT: 0.22129783693843594
Reg A MLP: 0.3261231281198003
Reg A SVM: 0.30782029950083195
Reg A DT: 0.2096505823627288


### **VA Regression from chunks**

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn import preprocessing

preprocessing.scale(vanilla_z_v)
preprocessing.scale(vanilla_z_a)
preprocessing.scale(reg_z_v)
preprocessing.scale(reg_z_a)

################# Vanilla Valence #########################

X_train, X_test, y_train, y_test = train_test_split(vanilla_z_v, vanilla_v_labels, test_size=0.3, random_state=42)
clf = Ridge()
clf.fit(X_train, y_train)
print('Vanilla V Ridge EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Vanilla V Ridge R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = Lasso()
clf.fit(X_train, y_train)
print('Vanilla V Lasso EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Vanilla V Lasso R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = SVR()
clf.fit(X_train, y_train)
print('Vanilla V SVR EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Vanilla V SVR R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = MLPRegressor()
clf.fit(X_train, y_train)
print('Vanilla V MLP EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Vanilla V MLP R2: ', str(r2_score(y_test, clf.predict(X_test))))

################# Vanilla Arousal #########################

X_train, X_test, y_train, y_test = train_test_split(vanilla_z_a, vanilla_a_labels, test_size=0.3, random_state=42)
clf = Ridge()
clf.fit(X_train, y_train)
print('Vanilla A Ridge EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Vanilla A Ridge R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = Lasso()
clf.fit(X_train, y_train)
print('Vanilla A Lasso EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Vanilla A Lasso R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = SVR()
clf.fit(X_train, y_train)
print('Vanilla A SVR EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Vanilla A SVR R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = MLPRegressor()
clf.fit(X_train, y_train)
print('Vanilla A MLP EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Vanilla A MLP R2: ', str(r2_score(y_test, clf.predict(X_test))))

################# Reg Valence #########################

X_train, X_test, y_train, y_test = train_test_split(reg_z_v, reg_v_labels, test_size=0.3, random_state=42)
clf = Ridge()
clf.fit(X_train, y_train)
print('Reg V Ridge EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Reg V Ridge R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = Lasso()
clf.fit(X_train, y_train)
print('Reg V Lasso EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Reg V Lasso R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = SVR()
clf.fit(X_train, y_train)
print('Reg V SVR EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Reg V SVR R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = MLPRegressor()
clf.fit(X_train, y_train)
print('Reg V MLP EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Reg V MLP R2: ', str(r2_score(y_test, clf.predict(X_test))))

################# Reg Arousal #########################

X_train, X_test, y_train, y_test = train_test_split(reg_z_a, reg_a_labels, test_size=0.3, random_state=42)
clf = Ridge()
clf.fit(X_train, y_train)
print('Reg A Ridge EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Reg A Ridge R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = Lasso()
clf.fit(X_train, y_train)
print('Reg A Lasso EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Reg A Lasso R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = SVR()
clf.fit(X_train, y_train)
print('Reg A SVR EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Reg A SVR R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = MLPRegressor()
clf.fit(X_train, y_train)
print('Reg A MLP EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Reg A MLP R2: ', str(r2_score(y_test, clf.predict(X_test))))

Vanilla V Ridge EV:  0.0052713444149676825
Vanilla V Ridge R2:  0.004660805116062372
Vanilla V Lasso EV:  0.0
Vanilla V Lasso R2:  -0.0015257252298581658
Vanilla V SVR EV:  -0.05015841834424828
Vanilla V SVR R2:  -0.05709585381138371
Vanilla V MLP EV:  -0.12500785116557944
Vanilla V MLP R2:  -0.23783968057054783
Vanilla A Ridge EV:  -0.015768044405738202
Vanilla A Ridge R2:  -0.01813083779573388
Vanilla A Lasso EV:  3.3306690738754696e-16
Vanilla A Lasso R2:  -0.001883777520963914
Vanilla A SVR EV:  -0.06899269791046203
Vanilla A SVR R2:  -0.07539708906590303
Vanilla A MLP EV:  -0.2516667635742942
Vanilla A MLP R2:  -0.2706584295615768
Reg V Ridge EV:  0.03706575636057674
Reg V Ridge R2:  0.036764119306737175
Reg V Lasso EV:  0.0
Reg V Lasso R2:  -0.0023175175286744842
Reg V SVR EV:  0.019733032791995897
Reg V SVR R2:  0.01964568325485383
Reg V MLP EV:  0.036181149114400624
Reg V MLP R2:  0.02572187702636186
Reg A Ridge EV:  0.009020164165458588
Reg A Ridge R2:  0.008923425093414039
Re

### **Inverse V -> A and A -> V regression**

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn import preprocessing

preprocessing.scale(vanilla_z_v)
preprocessing.scale(vanilla_z_a)
preprocessing.scale(reg_z_v)
preprocessing.scale(reg_z_a)

################# Vanilla Valence #########################

X_train, X_test, y_train, y_test = train_test_split(vanilla_z_v, vanilla_a_labels, test_size=0.3, random_state=42)
clf = Ridge()
clf.fit(X_train, y_train)
print('Vanilla V Ridge EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Vanilla V Ridge R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = Lasso()
clf.fit(X_train, y_train)
print('Vanilla V Lasso EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Vanilla V Lasso R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = SVR()
clf.fit(X_train, y_train)
print('Vanilla V SVR EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Vanilla V SVR R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = MLPRegressor()
clf.fit(X_train, y_train)
print('Vanilla V MLP EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Vanilla V MLP R2: ', str(r2_score(y_test, clf.predict(X_test))))

################# Vanilla Arousal #########################

X_train, X_test, y_train, y_test = train_test_split(vanilla_z_a, vanilla_v_labels, test_size=0.3, random_state=42)
clf = Ridge()
clf.fit(X_train, y_train)
print('Vanilla A Ridge EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Vanilla A Ridge R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = Lasso()
clf.fit(X_train, y_train)
print('Vanilla A Lasso EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Vanilla A Lasso R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = SVR()
clf.fit(X_train, y_train)
print('Vanilla A SVR EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Vanilla A SVR R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = MLPRegressor()
clf.fit(X_train, y_train)
print('Vanilla A MLP EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Vanilla A MLP R2: ', str(r2_score(y_test, clf.predict(X_test))))

################# Reg Valence #########################

X_train, X_test, y_train, y_test = train_test_split(reg_z_v, reg_a_labels, test_size=0.3, random_state=42)
clf = Ridge()
clf.fit(X_train, y_train)
print('Reg V Ridge EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Reg V Ridge R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = Lasso()
clf.fit(X_train, y_train)
print('Reg V Lasso EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Reg V Lasso R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = SVR()
clf.fit(X_train, y_train)
print('Reg V SVR EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Reg V SVR R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = MLPRegressor()
clf.fit(X_train, y_train)
print('Reg V MLP EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Reg V MLP R2: ', str(r2_score(y_test, clf.predict(X_test))))

################# Reg Arousal #########################

X_train, X_test, y_train, y_test = train_test_split(reg_z_a, reg_v_labels, test_size=0.3, random_state=42)
clf = Ridge()
clf.fit(X_train, y_train)
print('Reg A Ridge EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Reg A Ridge R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = Lasso()
clf.fit(X_train, y_train)
print('Reg A Lasso EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Reg A Lasso R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = SVR()
clf.fit(X_train, y_train)
print('Reg A SVR EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Reg A SVR R2: ', str(r2_score(y_test, clf.predict(X_test))))

clf = MLPRegressor()
clf.fit(X_train, y_train)
print('Reg A MLP EV: ', str(explained_variance_score(y_test, clf.predict(X_test))))
print('Reg A MLP R2: ', str(r2_score(y_test, clf.predict(X_test))))

Vanilla V Ridge EV:  -0.02280792494669215
Vanilla V Ridge R2:  -0.024122636006557174
Vanilla V Lasso EV:  3.3306690738754696e-16
Vanilla V Lasso R2:  -0.001883777520963914
Vanilla V SVR EV:  -0.07862687666377477
Vanilla V SVR R2:  -0.09448143697698974
Vanilla V MLP EV:  -0.2263121974770239
Vanilla V MLP R2:  -0.25128916097877574
Vanilla A Ridge EV:  -0.010995221804341204
Vanilla A Ridge R2:  -0.01218769866846059
Vanilla A Lasso EV:  0.0
Vanilla A Lasso R2:  -0.0015257252298581658
Vanilla A SVR EV:  -0.11695480607762843
Vanilla A SVR R2:  -0.13202755524808407
Vanilla A MLP EV:  -0.1906726419320972
Vanilla A MLP R2:  -0.2133612214176448
Reg V Ridge EV:  -0.015158674425295615
Reg V Ridge R2:  -0.015645889928405987
Reg V Lasso EV:  0.0
Reg V Lasso R2:  -0.00017974051525526669
Reg V SVR EV:  -0.07240956645008279
Reg V SVR R2:  -0.08635213733005487
Reg V MLP EV:  -0.045467606887044765
Reg V MLP R2:  -0.04590640243702904
Reg A Ridge EV:  -0.0015891989050140776
Reg A Ridge R2:  -0.003044964022

## **Qualitative Comparison**

### **TSNE plots**

In [0]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

data_dir = '/content/drive/My Drive/AFC_Project_data/IMFDB_data/'

vanilla_v_tsne = TSNE(n_components=2, perplexity=100).fit_transform(vanilla_z_v)
vanilla_a_tsne = TSNE(n_components=2, perplexity=100).fit_transform(vanilla_z_a)

colours = ['blue', 'green', 'yellow', 'red', 'black', 'brown', 'cyan', 'darkorange', 'mediumpurple', 'darkgrey']

plt.figure()
# ax = fig.add_subplot(111, projection='3d')

for i in range(len(vanilla_v_tsne)):
	plt.scatter(vanilla_v_tsne[i][0], vanilla_v_tsne[i][1], c=colours[vanilla_emotion_labels[i]], marker='o', s=4)
 
for i in range(len(vanilla_a_tsne)):
	plt.scatter(vanilla_a_tsne[i][0], vanilla_a_tsne[i][1], c=colours[vanilla_emotion_labels[i]], marker='.', s=4)
plt.savefig(data_dir+'tsne_vanilla.png')

reg_v_tsne = TSNE(n_components=2, perplexity=100).fit_transform(reg_z_v)
reg_a_tsne = TSNE(n_components=2, perplexity=100).fit_transform(reg_z_a)

plt.figure()
# ax = fig.add_subplot(111, projection='3d')

for i in range(len(reg_v_tsne)):
	plt.scatter(reg_v_tsne[i][0], reg_v_tsne[i][1], c=colours[reg_emotion_labels[i]], marker='o', s=4)
 
for i in range(len(reg_a_tsne)):
	plt.scatter(reg_a_tsne[i][0], reg_a_tsne[i][1], c=colours[reg_emotion_labels[i]], marker='.', s=4)
plt.savefig(data_dir+'tsne_reg.png')  

### **Latent Image-Chunk Comparison**

In [0]:
encoder = Encoder()
encoder.apply(weights_init)
encoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/IMFDB_data/encoder'))

decoder = Decoder()
decoder.apply(weights_init)
decoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/IMFDB_data/decoder'))

reg_encoder = IMFDB_Reg_Encoder()
reg_encoder.apply(weights_init)
reg_encoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/IMFDB_data/reg_encoder'))

reg_decoder = IMFDB_Reg_Decoder()
reg_decoder.apply(weights_init)
reg_decoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/IMFDB_data/reg_decoder'))

batch_size = 64

print('Loading Dataset...')
data_dir = '/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/'
transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])
dset = ImageFolderWithPathsIMFDB(data_dir, transform=transform)
print(dset.imgs[:10])
data_object = torch.utils.data.DataLoader(dset, batch_size=batch_size, shuffle=True, drop_last=True)

X1 = torch.zeros(batch_size, 3, 64, 64)

if torch.cuda.is_available():
  encoder.cuda()
  decoder.cuda()
  reg_encoder.cuda()
  reg_decoder.cuda()  
  X1 = X1.cuda()

ground_truths_v = []
ground_truths_a = []
regs_v = []
regs_a = []
vanillas_v = []
vanillas_a = []

with torch.no_grad():
  for iteration, (data_, _, labels, emotions) in tqdm(enumerate(data_object)):
      image_batch_1 = data_
      X1.copy_(image_batch_1)

      print(X1.shape)

      vanilla_z_v, _, _, vanilla_z_a, _, _, _, _, _ = encoder(X1)
      reg_z_v, _, _, reg_z_a, _, _, _, _, _ = reg_encoder(X1)

      vanilla_v_tsne = TSNE(n_components=2, perplexity=100).fit_transform(vanilla_z_v.detach().cpu().numpy())
      vanilla_a_tsne = TSNE(n_components=2, perplexity=100).fit_transform(vanilla_z_a.detach().cpu().numpy())

      reg_v_tsne = TSNE(n_components=2, perplexity=100).fit_transform(reg_z_v.detach().cpu().numpy())
      reg_a_tsne = TSNE(n_components=2, perplexity=100).fit_transform(reg_z_a.detach().cpu().numpy())

      ground_truths_v = ground_truths_v + list(labels[0].detach().cpu().numpy())
      ground_truths_a = ground_truths_a + list(labels[1].detach().cpu().numpy())

      regs_v = regs_v + list(np.mean(reg_v_tsne, axis=1)/100)
      regs_a = regs_a + list(np.mean(reg_a_tsne, axis=1)/100)

      vanillas_v = vanillas_v + list(np.mean(vanilla_v_tsne, axis=1)/25)
      vanillas_a = vanillas_a + list(np.mean(vanilla_a_tsne, axis=1)/25)

      if(iteration==0):
          break

data_dir = '/content/drive/My Drive/AFC_Project_data/IMFDB_data/'

plt.figure()
# ax = fig.add_subplot(111, projection='3d')

plt.scatter(ground_truths_v[0], ground_truths_a[0], c='red', marker='o', s=12, label='GT')
for i in range(1, batch_size):
	plt.scatter(ground_truths_v[i], ground_truths_a[i], c='red', marker='o', s=12)

plt.scatter(vanillas_v[0], vanillas_a[0], c='blue', marker='o', s=12, label='Vanilla VAE') 
for i in range(1, batch_size):
	plt.scatter(vanillas_v[i], vanillas_a[i], c='blue', marker='o', s=12)
 
plt.scatter(regs_v[0], regs_a[0], c='green', marker='o', s=12, label='Reg VAE')
for i in range(1, batch_size):
	plt.scatter(regs_v[i], regs_a[i], c='green', marker='o', s=12)

plt.legend(loc='best') 
plt.savefig(data_dir+'tsne_latent_comparison.png')
plt.close()

Loading Dataset...
[('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_1.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_10.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_11.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_12.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_13.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_14.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_15.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_16.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_17.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_ima

torch.Size([64, 3, 64, 64])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:72: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


### **Latent Traversal**

In [33]:
encoder = IMFDB_Reg_Encoder()
encoder.apply(weights_init)
encoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/IMFDB_data/encoder'))

decoder = IMFDB_Reg_Decoder()
decoder.apply(weights_init)
decoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/IMFDB_data/decoder'))

batch_size = 2

print('Loading Dataset...')
data_dir = '/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/'
transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])
dset = ImageFolderWithPathsIMFDB(data_dir, transform=transform)
print(dset.imgs[:10])
data_object = torch.utils.data.DataLoader(dset, batch_size=batch_size, shuffle=True, drop_last=True)

X1 = torch.zeros(batch_size, 3, 64, 64)

if torch.cuda.is_available():
  encoder.cuda()
  decoder.cuda()
  X1 = X1.cuda()

with torch.no_grad():
  for iteration, (data_, _, labels, emotions) in tqdm(enumerate(data_object)):
      image_batch_1 = data_
      X1.copy_(image_batch_1)
      z_v, mu_v, logvar_v, z_a, mu_a, logvar_a, z_z, mu_z, logvar_z = encoder(X1)

      img1 = X1[0]
      img2 = X1[1]

      print(img1.shape, img2.shape)

      ### V traversal with A0 and Z0 ###
      inter = np.zeros((10, z_v[0].shape[0]))
      z_a_new = z_a[0].repeat(10, 1)
      z_z_new = z_z[0].repeat(10, 1)

      print(z_a_new.shape, z_z_new.shape)
      for i in range(10):
          inter[i] = (z_v[0].detach().cpu().numpy() + i/10*(z_v[1].detach().cpu().numpy()-z_v[0].detach().cpu().numpy()))

      print(inter[0].shape)
      z_v_new = torch.FloatTensor(inter).cuda()
      print(z_v_new.shape)

      recon = decoder(z_v_new, z_a_new, z_z_new)
      save_image(recon[:10, :, :, :], '/content/drive/My Drive/AFC_Project_data/IMFDB_data/latent_traversal_v.png', nrow=2, normalize=True)

      ### A traversal with V0 and Z0 ###
      inter = np.zeros((10, z_a[0].shape[0]))
      z_v_new = z_v[0].repeat(10, 1)
      z_z_new = z_z[0].repeat(10, 1)

      print(z_v_new.shape, z_z_new.shape)
      for i in range(10):
          inter[i] = (z_a[0].detach().cpu().numpy() + i/10*(z_a[1].detach().cpu().numpy()-z_a[0].detach().cpu().numpy()))

      print(inter[0].shape)
      z_a_new = torch.FloatTensor(inter).cuda()
      print(z_a_new.shape)

      recon = decoder(z_v_new, z_a_new, z_z_new)
      save_image(recon[:10, :, :, :], '/content/drive/My Drive/AFC_Project_data/IMFDB_data/latent_traversal_a.png', nrow=2, normalize=True)

      ### Z traversal with V0 and Z0 ###
      inter = np.zeros((10, z_a[0].shape[0]))
      z_v_new = z_v[0].repeat(10, 1)
      z_a_new = z_a[0].repeat(10, 1)

      print(z_v_new.shape, z_a_new.shape)
      for i in range(10):
          inter[i] = (z_z[0].detach().cpu().numpy() + i/10*(z_z[1].detach().cpu().numpy()-z_z[0].detach().cpu().numpy()))

      print(inter[0].shape)
      z_z_new = torch.FloatTensor(inter).cuda()
      print(z_z_new.shape)

      recon = decoder(z_v_new, z_a_new, z_z_new)
      save_image(recon[:10, :, :, :], '/content/drive/My Drive/AFC_Project_data/IMFDB_data/latent_traversal_z.png', nrow=2, normalize=True)

      if(iteration==0):
          break

Loading Dataset...
[('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_1.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_10.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_11.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_12.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_13.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_14.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_15.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_16.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_17.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_ima

torch.Size([3, 64, 64]) torch.Size([3, 64, 64])
torch.Size([10, 32]) torch.Size([10, 32])
(32,)
torch.Size([10, 32])
torch.Size([10, 32]) torch.Size([10, 32])
(32,)
torch.Size([10, 32])
torch.Size([10, 32]) torch.Size([10, 32])
(32,)
torch.Size([10, 32])


# **IMFDB Discrete VA analysis**

In [11]:
os.chdir('/content/drive/My Drive/AFC_Project_data/IMFDB_data/processed_images')
!ls

AamairKhan	 Innocent	   Mamootty	       SharmilaTagore
Aarthi		 Jagadeesh	   Mohanlal	       SharukhKhan
AkshayKumar	 JagapathiBabu	   M.S.Narayana        Shashikumar
Ali		 Jagathi	   Mukesh	       ShilpaShetty
allimages	 Jamuna		   Nagarjuna	       Shivaram
Ambresh		 JayaBhaduri	   NedumudiVenu        Shobana
AmitabhBachchan  Jayaprakash	   NTR		       Siddique
AmrishPuri	 Jayram		   PareshRaval	       Simran
AnilKapoor	 JosePrakash	   Pavithralokesh      SoumithraChatterjee
Annapoorna	 KajalAgarwal	   PrakashRaj	       Soundarya
ANR		 Kajol		   Prema	       Srihari
AnupamKher	 KareenaKapoor	   PremaNazir	       SureshGopi
Ashwath		 KarunaBenerjee    RajeshKhanna        Suryakantham
Avinash		 KatrinaKaif	   RakheeGulzar        SVR
BabuMohan	 KavyaMadhavan	   RamanaReddy	       TanikellaBharani
Balakrishna	 KotaSrinivasarao  Ramaprabha	       Tenniskrishna
Bhanupriya	 K.Viswanath	   RameshArvind        Thilakan
Bharathi	 Lakshmidevi	   Rameshbhatt	       Trisha
BomanIrani	 Leelavathi	 

In [12]:
  import json

  json_file = '/content/drive/My Drive/AFC_Project_data/IMFDB_data/IMFDB_va_discrete.json'

  with open(json_file) as f:
    va_dictionary = json.load(f)
  print(va_dictionary) 

import pandas as pd
df = pd.DataFrame(list(va_dictionary.values()), columns=['Valence', 'Arousal'])
# print(df.head)

plt.figure()
df.plot.box(grid='True')
plt.savefig('/content/drive/My Drive/AFC_Project_data/IMFDB_data/discrete_va_labels.png')
plt.close()

{'Gundammakatha_ANR_100.jpg': [-1, 1], 'Gundammakatha_ANR_101.jpg': [-1, 0], 'Gundammakatha_ANR_102.jpg': [-1, 0], 'Gundammakatha_ANR_103.jpg': [8, 0], 'Gundammakatha_ANR_104.jpg': [0, -1], 'Gundammakatha_ANR_105.jpg': [1, 0], 'Gundammakatha_ANR_106.jpg': [0, 0], 'Gundammakatha_ANR_107.jpg': [6, 2], 'Gundammakatha_ANR_11.jpg': [0, -1], 'Gundammakatha_ANR_111.jpg': [0, 0], 'Gundammakatha_ANR_113.jpg': [1, 1], 'Gundammakatha_ANR_117.jpg': [1, 8], 'Gundammakatha_ANR_118.jpg': [-2, -1], 'Gundammakatha_ANR_12.jpg': [0, 1], 'Gundammakatha_ANR_120.jpg': [1, 6], 'Gundammakatha_ANR_121.jpg': [0, 0], 'Gundammakatha_ANR_122.jpg': [-1, -1], 'Gundammakatha_ANR_123.jpg': [-7, -2], 'Gundammakatha_ANR_125.jpg': [1, 6], 'Gundammakatha_ANR_126.jpg': [1, 8], 'Gundammakatha_ANR_127.jpg': [8, 0], 'Gundammakatha_ANR_128.jpg': [6, 2], 'Gundammakatha_ANR_129.jpg': [3, 6], 'Gundammakatha_ANR_130.jpg': [7, 1], 'Gundammakatha_ANR_131.jpg': [-7, -2], 'Gundammakatha_ANR_132.jpg': [-7, -2], 'Gundammakatha_ANR_133.j

In [0]:
emotion_mapping = ['ANGER', 'DISGUST', 'FEAR', 'HAPPINESS', 'NEUTRAL', 'SADNESS', 'SURPRISE']

class ImageFolderWithPathsIMFDBDiscrete(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPathsIMFDBDiscrete, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        # folder = path[-13:-10]
        fil = path[path.rfind('/')+1:]
        tuple_with_path = (original_tuple+(va_dictionary[fil],))
        return tuple_with_path

## **Quantitative Comparison**

In [36]:
encoder = IMFDB_Reg_Discrete_Encoder()
encoder.apply(weights_init)
encoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/IMFDB_data/reg_encoder'))

decoder = IMFDB_Reg_Discrete_Decoder()
decoder.apply(weights_init)
decoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/IMFDB_data/reg_decoder'))

batch_size = 2

print('Loading Dataset...')
data_dir = '/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/'
transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])
dset = ImageFolderWithPathsIMFDBDiscrete(data_dir, transform=transform)
print(dset.imgs[:10])
data_object = torch.utils.data.DataLoader(dset, batch_size=batch_size, shuffle=True, drop_last=True)

X1 = torch.zeros(batch_size, 3, 64, 64)

if torch.cuda.is_available():
  encoder.cuda()
  decoder.cuda()
  X1 = X1.cuda()

s_metric = 0
s_v_m = 0
s_a_m = 0

a_metric = 0
a_v_m = 0
a_a_m = 0

all_z_v = []
all_z_a = []

all_v_labels = []
all_a_labels = []
all_emotion_labels = []

all_l_v = []
all_l_a = []

with torch.no_grad():
  for iteration, (data_, _, labels) in tqdm(enumerate(data_object)):
      image_batch_1 = data_
      X1.copy_(image_batch_1)
      z_v, mu_v, logvar_v, z_a, mu_a, logvar_a, z_z, mu_z, logvar_z, l_v, l_a = encoder(X1)

      all_z_v = all_z_v + list(z_v.detach().cpu().numpy())
      all_z_a = all_z_a + list(z_a.detach().cpu().numpy())

      all_v_labels = all_v_labels + list(labels[0].detach().cpu().numpy())
      all_a_labels = all_a_labels + list(labels[1].detach().cpu().numpy())

      all_l_v = all_l_v + list(l_v.detach().cpu().numpy())
      all_l_a = all_l_a + list(l_a.detach().cpu().numpy())

      if(iteration==1000):
          break
  
import pickle
pickle.dump(all_l_v, open('/content/drive/My Drive/AFC_Project_data/IMFDB_data/reg_vae_l_v_discrete.pickle', 'wb'))
pickle.dump(all_l_a, open('/content/drive/My Drive/AFC_Project_data/IMFDB_data/reg_vae_l_a_discrete.pickle', 'wb'))
pickle.dump(all_z_v, open('/content/drive/My Drive/AFC_Project_data/IMFDB_data/reg_vae_z_v_discrete.pickle', 'wb'))
pickle.dump(all_z_a, open('/content/drive/My Drive/AFC_Project_data/IMFDB_data/reg_vae_z_a_discrete.pickle', 'wb'))
pickle.dump(all_v_labels, open('/content/drive/My Drive/AFC_Project_data/IMFDB_data/reg_vae_v_labels_discrete.pickle', 'wb'))
pickle.dump(all_a_labels, open('/content/drive/My Drive/AFC_Project_data/IMFDB_data/reg_vae_a_labels_discrete.pickle', 'wb'))

Loading Dataset...
[('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_1.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_10.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_11.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_12.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_13.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_14.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_15.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_16.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_17.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_ima

In [15]:
import pickle

data_dir = '/content/drive/My Drive/AFC_Project_data/IMFDB_data/'

reg_z_v_discrete = pickle.load(open(data_dir+'reg_vae_z_v_discrete.pickle', 'rb'))
reg_z_a_discrete = pickle.load(open(data_dir+'reg_vae_z_a_discrete.pickle', 'rb'))
reg_v_labels_discrete = pickle.load(open(data_dir+'reg_vae_v_labels_discrete.pickle', 'rb'))
reg_a_labels_discrete = pickle.load(open(data_dir+'reg_vae_a_labels_discrete.pickle', 'rb'))
reg_l_v_discrete = pickle.load(open(data_dir+'reg_vae_l_v_discrete.pickle', 'rb'))
reg_l_a_discrete = pickle.load(open(data_dir+'reg_vae_l_a_discrete.pickle', 'rb'))

vanilla_z_v = pickle.load(open(data_dir+'vanilla_vae_z_v.pickle', 'rb'))
vanilla_z_a = pickle.load(open(data_dir+'vanilla_vae_z_a.pickle', 'rb'))
vanilla_v_labels = pickle.load(open(data_dir+'vanilla_vae_v_labels.pickle', 'rb'))
vanilla_a_labels = pickle.load(open(data_dir+'vanilla_vae_a_labels.pickle', 'rb'))

print(len(reg_z_v_discrete), len(reg_z_a_discrete), len(reg_v_labels_discrete), len(reg_a_labels_discrete), len(reg_l_v_discrete), len(reg_l_a_discrete))
print(len(vanilla_z_v), len(vanilla_z_a), len(vanilla_v_labels), len(vanilla_a_labels))

2002 2002 2002 2002 2002 2002
2002 2002 2002 2002


### **VA Classification from chunks**

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier

preprocessing.scale(vanilla_z_v)
preprocessing.scale(vanilla_z_a)
preprocessing.scale(reg_z_v_discrete)
preprocessing.scale(reg_z_a_discrete)
preprocessing.scale(reg_l_v_discrete)
preprocessing.scale(reg_l_a_discrete)

######################################## Reg V ############################################

X_train, X_test, y_train, y_test = train_test_split(reg_z_v_discrete, reg_v_labels_discrete, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V MLP: '+str(accuracy_score(y_test, preds)))

clf = clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V SVM: '+str(accuracy_score(y_test, preds)))

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# preds = clf.predict(X_test)
# print('Reg V LR: '+str(accuracy_score(y_test, preds)))

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V DT: '+str(accuracy_score(y_test, preds)))

######################################## Reg A ############################################

X_train, X_test, y_train, y_test = train_test_split(reg_z_a_discrete, reg_a_labels_discrete, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A MLP: '+str(accuracy_score(y_test, preds)))

clf = clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A SVM: '+str(accuracy_score(y_test, preds)))

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# preds = clf.predict(X_test)
# print('Reg A LR: '+str(accuracy_score(y_test, preds)))

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A DT: '+str(accuracy_score(y_test, preds)))

######################################## Reg projection V ############################################

X_train, X_test, y_train, y_test = train_test_split(reg_l_v_discrete, reg_v_labels_discrete, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V MLP: '+str(accuracy_score(y_test, preds)))

clf = clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V SVM: '+str(accuracy_score(y_test, preds)))

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# preds = clf.predict(X_test)
# print('Reg V LR: '+str(accuracy_score(y_test, preds)))

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V DT: '+str(accuracy_score(y_test, preds)))

######################################## Reg projection A ############################################

X_train, X_test, y_train, y_test = train_test_split(reg_l_a_discrete, reg_a_labels_discrete, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A MLP: '+str(accuracy_score(y_test, preds)))

clf = clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A SVM: '+str(accuracy_score(y_test, preds)))

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# preds = clf.predict(X_test)
# print('Reg A LR: '+str(accuracy_score(y_test, preds)))

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A DT: '+str(accuracy_score(y_test, preds)))

Reg V MLP: 0.10815307820299501
Reg V SVM: 0.11980033277870217
Reg V DT: 0.07820299500831947
Reg A MLP: 0.16805324459234608
Reg A SVM: 0.2129783693843594
Reg A DT: 0.12479201331114809
Reg V MLP: 0.08818635607321132
Reg V SVM: 0.09983361064891846
Reg V DT: 0.07820299500831947
Reg A MLP: 0.10815307820299501
Reg A SVM: 0.2129783693843594
Reg A DT: 0.12312811980033278


### **Inverse V->A and A->V classification**

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier

preprocessing.scale(vanilla_z_v)
preprocessing.scale(vanilla_z_a)
preprocessing.scale(reg_z_v_discrete)
preprocessing.scale(reg_z_a_discrete)

######################################## Reg V->A ############################################

X_train, X_test, y_train, y_test = train_test_split(reg_z_v_discrete, reg_a_labels_discrete, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V MLP: '+str(accuracy_score(y_test, preds)))

clf = clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V SVM: '+str(accuracy_score(y_test, preds)))

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# preds = clf.predict(X_test)
# print('Reg V LR: '+str(accuracy_score(y_test, preds)))

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V DT: '+str(accuracy_score(y_test, preds)))

######################################## Reg A->V ############################################

X_train, X_test, y_train, y_test = train_test_split(reg_z_a_discrete, reg_v_labels_discrete, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A MLP: '+str(accuracy_score(y_test, preds)))

clf = clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A SVM: '+str(accuracy_score(y_test, preds)))

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# preds = clf.predict(X_test)
# print('Reg A LR: '+str(accuracy_score(y_test, preds)))

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A DT: '+str(accuracy_score(y_test, preds)))

Reg V MLP: 0.16472545757071547
Reg V SVM: 0.2129783693843594
Reg V DT: 0.11813643926788686
Reg A MLP: 0.10316139767054909
Reg A SVM: 0.10981697171381032
Reg A DT: 0.07487520798668885


## **Qualitative Comparison**

### **Latent Image-Chunk comparison**

In [26]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

reg_encoder = IMFDB_Reg_Discrete_Encoder()
reg_encoder.apply(weights_init)
reg_encoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/IMFDB_data/reg_encoder'))

reg_decoder = IMFDB_Reg_Discrete_Decoder()
reg_decoder.apply(weights_init)
reg_decoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/IMFDB_data/reg_decoder'))

encoder = Encoder()
encoder.apply(weights_init)
encoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/IMFDB_data/encoder'))

decoder = Decoder()
decoder.apply(weights_init)
decoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/IMFDB_data/decoder'))

batch_size = 64

print('Loading Dataset...')
data_dir = '/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/'
transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])
dset = ImageFolderWithPathsIMFDBDiscrete(data_dir, transform=transform)
print(dset.imgs[:10])
data_object = torch.utils.data.DataLoader(dset, batch_size=batch_size, shuffle=True, drop_last=True)

X1 = torch.zeros(batch_size, 3, 64, 64)

if torch.cuda.is_available():
  encoder.cuda()
  decoder.cuda()
  reg_encoder.cuda()
  reg_decoder.cuda()  
  X1 = X1.cuda()

ground_truths_v = []
ground_truths_a = []
regs_v = []
regs_a = []
vanillas_v = []
vanillas_a = []

with torch.no_grad():
  for iteration, (data_, _, labels) in tqdm(enumerate(data_object)):
      image_batch_1 = data_
      X1.copy_(image_batch_1)

      print(X1.shape)

      vanilla_z_v, _, _, vanilla_z_a, _, _, _, _, _ = encoder(X1)
      reg_z_v, _, _, reg_z_a, _, _, _, _, _, _, _ = reg_encoder(X1)

      vanilla_v_tsne = TSNE(n_components=2, perplexity=100).fit_transform(vanilla_z_v.detach().cpu().numpy())
      vanilla_a_tsne = TSNE(n_components=2, perplexity=100).fit_transform(vanilla_z_a.detach().cpu().numpy())

      reg_v_tsne = TSNE(n_components=2, perplexity=100).fit_transform(reg_z_v.detach().cpu().numpy())
      reg_a_tsne = TSNE(n_components=2, perplexity=100).fit_transform(reg_z_a.detach().cpu().numpy())

      ground_truths_v = ground_truths_v + list(labels[0].detach().cpu().numpy())
      ground_truths_a = ground_truths_a + list(labels[1].detach().cpu().numpy())

      regs_v = regs_v + list(np.mean(reg_v_tsne, axis=1)/25)
      regs_a = regs_a + list(np.mean(reg_a_tsne, axis=1)/25)

      vanillas_v = vanillas_v + list(np.mean(vanilla_v_tsne, axis=1))
      vanillas_a = vanillas_a + list(np.mean(vanilla_a_tsne, axis=1))

      if(iteration==0):
          break

data_dir = '/content/drive/My Drive/AFC_Project_data/IMFDB_data/'

plt.figure()
# ax = fig.add_subplot(111, projection='3d')

plt.scatter(ground_truths_v[0], ground_truths_a[0], c='red', marker='o', s=12, label='GT')
for i in range(1, batch_size):
	plt.scatter(ground_truths_v[i], ground_truths_a[i], c='red', marker='o', s=12)

plt.scatter(vanillas_v[0], vanillas_a[0], c='blue', marker='o', s=12, label='Vanilla VAE') 
for i in range(1, batch_size):
	plt.scatter(vanillas_v[i], vanillas_a[i], c='blue', marker='o', s=12)
 
plt.scatter(regs_v[0], regs_a[0], c='green', marker='o', s=12, label='Reg VAE')
for i in range(1, batch_size):
	plt.scatter(regs_v[i], regs_a[i], c='green', marker='o', s=12)

plt.legend(loc='best') 
plt.savefig(data_dir+'tsne_latent_discrete_comparison.png')
plt.close()

Loading Dataset...
[('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_1.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_10.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_11.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_12.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_13.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_14.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_15.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_16.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_images/train/3Idiots_Kareena_17.jpg', 0), ('/content/drive/My Drive/AFC_Project_data/IMFDB_data/final_ima

torch.Size([64, 3, 64, 64])


# **AFEW analysis**

In [0]:
import json

json_file = '/content/drive/My Drive/AFC_Project_data/AFEW_data/AFEW_va.json'

with open(json_file) as f:
  va_dictionary = json.load(f)
print(va_dictionary) 

{'001_00000': [0.0, 5.0], '001_00001': [0.0, 5.0], '001_00002': [0.0, 5.0], '001_00003': [0.0, 5.0], '001_00004': [0.0, 5.0], '001_00005': [0.0, 5.0], '001_00006': [0.0, 5.0], '001_00007': [0.0, 5.0], '001_00008': [0.0, 5.0], '001_00009': [0.0, 5.0], '001_00010': [0.0, 5.0], '001_00011': [0.0, 5.0], '001_00012': [0.0, 5.0], '001_00013': [0.0, 5.0], '001_00014': [0.0, 5.0], '001_00015': [0.0, 5.0], '001_00016': [0.0, 5.0], '001_00017': [0.0, 5.0], '001_00018': [0.0, 5.0], '001_00019': [0.0, 5.0], '001_00020': [0.0, 5.0], '001_00021': [0.0, 5.0], '001_00022': [0.0, 5.0], '001_00023': [0.0, 5.0], '001_00024': [0.0, 5.0], '001_00025': [0.0, 5.0], '001_00026': [0.0, 5.0], '001_00027': [0.0, 5.0], '001_00028': [0.0, 5.0], '001_00029': [0.0, 5.0], '001_00030': [0.0, 5.0], '001_00031': [0.0, 5.0], '001_00032': [0.0, 5.0], '001_00033': [0.0, 5.0], '001_00034': [0.0, 5.0], '001_00035': [0.0, 5.0], '001_00036': [0.0, 5.0], '001_00037': [0.0, 5.0], '001_00038': [0.0, 5.0], '001_00039': [0.0, 5.0],

In [0]:
class ImageFolderWithPathsAFEW(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPathsAFEW, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        folder = path[-13:-10]
        fil = path[-9:-4]
        tuple_with_path = (original_tuple+(va_dictionary[folder+'_'+fil],))
        return tuple_with_path

## **Quantitative Comparison**

In [0]:
encoder = Encoder()
encoder.apply(weights_init)
encoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/AFEW_data/encoder'))

decoder = Decoder()
decoder.apply(weights_init)
decoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/AFEW_data/decoder'))

batch_size = 2

print('Loading Dataset...')
data_dir = '/content/drive/My Drive/AFC_Project_data/AFEW_data/processed_images/'
transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])
dset = ImageFolderWithPathsAFEW(data_dir, transform=transform)
data_object = torch.utils.data.DataLoader(dset, batch_size=batch_size, shuffle=True, drop_last=True)

X1 = torch.zeros(batch_size, 3, 64, 64)

if torch.cuda.is_available():
  encoder.cuda()
  decoder.cuda()
  X1 = X1.cuda()

s_metric = 0
s_v_m = 0
s_a_m = 0

a_metric = 0
a_v_m = 0
a_a_m = 0

all_z_v = []
all_z_a = []

all_v_labels = []
all_a_labels = []
all_l_v = []
all_l_a = []

with torch.no_grad():
  for iteration, (data_, _, labels) in tqdm(enumerate(data_object)):
      z_v, mu_v, logvar_v, z_a, mu_a, logvar_a, z_z, mu_z, logvar_z = encoder(X1)

      all_z_v = all_z_v + list(z_v.detach().cpu().numpy())
      all_z_a = all_z_a + list(z_a.detach().cpu().numpy())

      all_v_labels = all_v_labels + list(labels[0].detach().cpu().numpy())
      all_a_labels = all_a_labels + list(labels[1].detach().cpu().numpy())

      if(iteration==1000):
          break
  
import pickle

pickle.dump(all_z_v, open('/content/drive/My Drive/AFC_Project_data/AFEW_data/vanilla_vae_z_v.pickle', 'wb'))
pickle.dump(all_z_a, open('/content/drive/My Drive/AFC_Project_data/AFEW_data/vanilla_vae_z_a.pickle', 'wb'))
pickle.dump(all_v_labels, open('/content/drive/My Drive/AFC_Project_data/AFEW_data/vanilla_vae_v_labels.pickle', 'wb'))
pickle.dump(all_a_labels, open('/content/drive/My Drive/AFC_Project_data/AFEW_data/vanilla_vae_a_labels.pickle', 'wb'))

Loading Dataset...


In [0]:
encoder = AFEW_Reg_Encoder()
encoder.apply(weights_init)
encoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/AFEW_data/reg_encoder'))

decoder = AFEW_Reg_Decoder()
decoder.apply(weights_init)
decoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/AFEW_data/reg_decoder'))

batch_size = 2

print('Loading Dataset...')
data_dir = '/content/drive/My Drive/AFC_Project_data/AFEW_data/processed_images/'
transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])
dset = ImageFolderWithPathsAFEW(data_dir, transform=transform)
data_object = torch.utils.data.DataLoader(dset, batch_size=batch_size, shuffle=True, drop_last=True)

X1 = torch.zeros(batch_size, 3, 64, 64)

if torch.cuda.is_available():
  encoder.cuda()
  decoder.cuda()
  X1 = X1.cuda()

s_metric = 0
s_v_m = 0
s_a_m = 0

a_metric = 0
a_v_m = 0
a_a_m = 0

all_z_v = []
all_z_a = []

all_v_labels = []
all_a_labels = []
all_l_v = []
all_l_a = []

with torch.no_grad():
  for iteration, (data_, _, labels) in tqdm(enumerate(data_object)):
      z_v, mu_v, logvar_v, z_a, mu_a, logvar_a, z_z, mu_z, logvar_z, l_v, l_a = encoder(X1)

      all_z_v = all_z_v + list(z_v.detach().cpu().numpy())
      all_z_a = all_z_a + list(z_a.detach().cpu().numpy())

      all_l_v = all_l_v + list(l_v.detach().cpu().numpy())
      all_l_a = all_l_a + list(l_a.detach().cpu().numpy())

      all_v_labels = all_v_labels + list(labels[0].detach().cpu().numpy())
      all_a_labels = all_a_labels + list(labels[1].detach().cpu().numpy())

      if(iteration==1000):
          break
  
import pickle

pickle.dump(all_z_v, open('/content/drive/My Drive/AFC_Project_data/AFEW_data/reg_vae_z_v.pickle', 'wb'))
pickle.dump(all_z_a, open('/content/drive/My Drive/AFC_Project_data/AFEW_data/reg_vae_z_a.pickle', 'wb'))
pickle.dump(all_l_v, open('/content/drive/My Drive/AFC_Project_data/AFEW_data/reg_vae_l_v.pickle', 'wb'))
pickle.dump(all_l_a, open('/content/drive/My Drive/AFC_Project_data/AFEW_data/reg_vae_l_a.pickle', 'wb'))
pickle.dump(all_v_labels, open('/content/drive/My Drive/AFC_Project_data/AFEW_data/reg_vae_v_labels.pickle', 'wb'))
pickle.dump(all_a_labels, open('/content/drive/My Drive/AFC_Project_data/AFEW_data/reg_vae_a_labels.pickle', 'wb'))

Loading Dataset...


In [11]:
import pickle

data_dir = '/content/drive/My Drive/AFC_Project_data/AFEW_data/'

reg_z_v = pickle.load(open(data_dir+'reg_vae_z_v.pickle', 'rb'))
reg_z_a = pickle.load(open(data_dir+'reg_vae_z_a.pickle', 'rb'))
reg_l_v = pickle.load(open(data_dir+'reg_vae_l_v.pickle', 'rb'))
reg_l_a = pickle.load(open(data_dir+'reg_vae_l_a.pickle', 'rb'))
reg_v_labels = pickle.load(open(data_dir+'reg_vae_v_labels.pickle', 'rb'))
reg_a_labels = pickle.load(open(data_dir+'reg_vae_a_labels.pickle', 'rb'))

vanilla_z_v = pickle.load(open(data_dir+'vanilla_vae_z_v.pickle', 'rb'))
vanilla_z_a = pickle.load(open(data_dir+'vanilla_vae_z_a.pickle', 'rb'))
vanilla_v_labels = pickle.load(open(data_dir+'vanilla_vae_v_labels.pickle', 'rb'))
vanilla_a_labels = pickle.load(open(data_dir+'vanilla_vae_a_labels.pickle', 'rb'))

print(len(reg_z_v), len(reg_z_a), len(reg_l_v), len(reg_l_a), len(reg_v_labels), len(reg_a_labels))
print(len(vanilla_z_v), len(vanilla_z_a), len(vanilla_v_labels), len(vanilla_a_labels))

2002 2002 2002 2002 2002 2002
2002 2002 2002 2002


### **VA Classification from chunks**

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier

preprocessing.scale(vanilla_z_v)
preprocessing.scale(vanilla_z_a)
preprocessing.scale(reg_z_v)
preprocessing.scale(reg_z_a)
preprocessing.scale(reg_l_v)
preprocessing.scale(reg_l_a)

######################################## Vanilla V ############################################

X_train, X_test, y_train, y_test = train_test_split(vanilla_z_v, vanilla_v_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla V MLP: '+str(accuracy_score(y_test, preds)))

clf = clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla V SVM: '+str(accuracy_score(y_test, preds)))

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# preds = clf.predict(X_test)
# print('Vanilla V LR: '+str(accuracy_score(y_test, preds)))

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla V DT: '+str(accuracy_score(y_test, preds)))

######################################## Vanilla A ############################################

X_train, X_test, y_train, y_test = train_test_split(vanilla_z_a, vanilla_a_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla A MLP: '+str(accuracy_score(y_test, preds)))

clf = clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla A SVM: '+str(accuracy_score(y_test, preds)))

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# preds = clf.predict(X_test)
# print('Vanilla A LR: '+str(accuracy_score(y_test, preds)))

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla A DT: '+str(accuracy_score(y_test, preds)))

######################################## Reg V ############################################

X_train, X_test, y_train, y_test = train_test_split(reg_z_v, reg_v_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V MLP: '+str(accuracy_score(y_test, preds)))

clf = clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V SVM: '+str(accuracy_score(y_test, preds)))

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# preds = clf.predict(X_test)
# print('Reg V LR: '+str(accuracy_score(y_test, preds)))

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V DT: '+str(accuracy_score(y_test, preds)))

######################################## Reg A ############################################

X_train, X_test, y_train, y_test = train_test_split(reg_z_a, reg_a_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A MLP: '+str(accuracy_score(y_test, preds)))

clf = clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A SVM: '+str(accuracy_score(y_test, preds)))

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# preds = clf.predict(X_test)
# print('Reg A LR: '+str(accuracy_score(y_test, preds)))

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A DT: '+str(accuracy_score(y_test, preds)))

######################################## Reg projection V ############################################

X_train, X_test, y_train, y_test = train_test_split(reg_l_v, reg_v_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V MLP: '+str(accuracy_score(y_test, preds)))

clf = clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V SVM: '+str(accuracy_score(y_test, preds)))

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# preds = clf.predict(X_test)
# print('Reg V LR: '+str(accuracy_score(y_test, preds)))

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V DT: '+str(accuracy_score(y_test, preds)))

######################################## Reg projection A ############################################

X_train, X_test, y_train, y_test = train_test_split(reg_l_a, reg_a_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A MLP: '+str(accuracy_score(y_test, preds)))

clf = clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A SVM: '+str(accuracy_score(y_test, preds)))

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# preds = clf.predict(X_test)
# print('Reg A LR: '+str(accuracy_score(y_test, preds)))

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A DT: '+str(accuracy_score(y_test, preds)))

Vanilla V MLP: 0.2762063227953411
Vanilla V SVM: 0.2762063227953411
Vanilla V DT: 0.2762063227953411
Vanilla A MLP: 0.13477537437603992
Vanilla A SVM: 0.15307820299500832
Vanilla A DT: 0.15307820299500832
Reg V MLP: 0.28286189683860236
Reg V SVM: 0.28286189683860236
Reg V DT: 0.28286189683860236
Reg A MLP: 0.18136439267886856
Reg A SVM: 0.18136439267886856
Reg A DT: 0.18136439267886856
Reg V MLP: 0.2079866888519135
Reg V SVM: 0.28286189683860236
Reg V DT: 0.28286189683860236
Reg A MLP: 0.13810316139767054
Reg A SVM: 0.18136439267886856
Reg A DT: 0.18136439267886856


### **Inverse V->A and A->V classification**

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier

preprocessing.scale(vanilla_z_v)
preprocessing.scale(vanilla_z_a)
preprocessing.scale(reg_z_v)
preprocessing.scale(reg_z_a)

######################################## Vanilla V->A ############################################

X_train, X_test, y_train, y_test = train_test_split(vanilla_z_v, vanilla_a_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla V MLP: '+str(accuracy_score(y_test, preds)))

clf = clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla V SVM: '+str(accuracy_score(y_test, preds)))

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# preds = clf.predict(X_test)
# print('Vanilla V LR: '+str(accuracy_score(y_test, preds)))

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla V DT: '+str(accuracy_score(y_test, preds)))

######################################## Vanilla A->V ############################################

X_train, X_test, y_train, y_test = train_test_split(vanilla_z_a, vanilla_v_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla A MLP: '+str(accuracy_score(y_test, preds)))

clf = clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla A SVM: '+str(accuracy_score(y_test, preds)))

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# preds = clf.predict(X_test)
# print('Vanilla A LR: '+str(accuracy_score(y_test, preds)))

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Vanilla A DT: '+str(accuracy_score(y_test, preds)))

######################################## Reg V->A ############################################

X_train, X_test, y_train, y_test = train_test_split(reg_z_v, reg_a_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V MLP: '+str(accuracy_score(y_test, preds)))

clf = clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V SVM: '+str(accuracy_score(y_test, preds)))

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# preds = clf.predict(X_test)
# print('Reg V LR: '+str(accuracy_score(y_test, preds)))

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg V DT: '+str(accuracy_score(y_test, preds)))

######################################## Reg A->V ############################################

X_train, X_test, y_train, y_test = train_test_split(reg_z_a, reg_v_labels, test_size=0.3, random_state=42)

clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A MLP: '+str(accuracy_score(y_test, preds)))

clf = clf = svm.SVC()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A SVM: '+str(accuracy_score(y_test, preds)))

# clf = LogisticRegression()
# clf.fit(X_train, y_train)
# preds = clf.predict(X_test)
# print('Reg A LR: '+str(accuracy_score(y_test, preds)))

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('Reg A DT: '+str(accuracy_score(y_test, preds)))

Vanilla V MLP: 0.13477537437603992
Vanilla V SVM: 0.15307820299500832
Vanilla V DT: 0.15307820299500832
Vanilla A MLP: 0.2762063227953411
Vanilla A SVM: 0.2762063227953411
Vanilla A DT: 0.2762063227953411
Reg V MLP: 0.18136439267886856
Reg V SVM: 0.18136439267886856
Reg V DT: 0.18136439267886856
Reg A MLP: 0.28286189683860236
Reg A SVM: 0.28286189683860236
Reg A DT: 0.28286189683860236


## **Qualitative Comparison**

### **TSNE plots**

In [0]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

data_dir = '/content/drive/My Drive/AFC_Project_data/AFEW_data/'

vanilla_v_tsne = TSNE(n_components=2, perplexity=100).fit_transform(vanilla_z_v)
vanilla_a_tsne = TSNE(n_components=2, perplexity=100).fit_transform(vanilla_z_a)

# colours = ['blue', 'green', 'yellow', 'red', 'black', 'brown', 'cyan', 'darkorange', 'mediumpurple', 'darkgrey']
cmap = plt.get_cmap('jet')
colours = cmap(np.linspace(0, 1.0, 81))

plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# c=colours[int(vanilla_v_labels[i])+10]

for i in range(len(vanilla_v_tsne)):
	if(vanilla_v_labels[i]>0):
			plt.scatter(vanilla_v_tsne[i][0], vanilla_v_tsne[i][1], color='r', marker='o', s=4)
	else:
			plt.scatter(vanilla_v_tsne[i][0], vanilla_v_tsne[i][1], color='b', marker='o', s=4)

for i in range(len(vanilla_a_tsne)):
	if(vanilla_v_labels[i]>0):
			plt.scatter(vanilla_a_tsne[i][0], vanilla_a_tsne[i][1], color='r', marker='.', s=4)
	else:
			plt.scatter(vanilla_a_tsne[i][0], vanilla_a_tsne[i][1], color='b', marker='.', s=4)
plt.savefig(data_dir+'tsne_vanilla_labelled_v.png')

plt.figure()
# ax = fig.add_subplot(111, projection='3d')

for i in range(len(vanilla_v_tsne)):
	if(vanilla_a_labels[i]>0):
		plt.scatter(vanilla_v_tsne[i][0], vanilla_v_tsne[i][1], color='r', marker='o', s=4)
	else:
		plt.scatter(vanilla_v_tsne[i][0], vanilla_v_tsne[i][1], color='b', marker='o', s=4)
 
for i in range(len(vanilla_a_tsne)):
	if(vanilla_a_labels[i]>0):
			plt.scatter(vanilla_a_tsne[i][0], vanilla_a_tsne[i][1], color='r', marker='.', s=4)
	else:
			plt.scatter(vanilla_a_tsne[i][1], vanilla_a_tsne[i][1], color='b', marker='.', s=4)
plt.savefig(data_dir+'tsne_vanilla_labelled_a.png')

reg_v_tsne = TSNE(n_components=2, perplexity=100).fit_transform(reg_z_v)
reg_a_tsne = TSNE(n_components=2, perplexity=100).fit_transform(reg_z_a)

plt.figure()
# ax = fig.add_subplot(111, projection='3d')

for i in range(len(reg_v_tsne)):
	if(reg_v_labels[i]>0):
		plt.scatter(reg_v_tsne[i][0], reg_v_tsne[i][1], color='r', marker='o', s=4)
	else:
		plt.scatter(reg_v_tsne[i][0], reg_v_tsne[i][1], color='b', marker='o', s=4)
 
for i in range(len(reg_a_tsne)):
	if(reg_v_labels[i]>0):
		plt.scatter(reg_a_tsne[i][0], reg_a_tsne[i][1], color='r', marker='.', s=4)
	else:
		plt.scatter(reg_a_tsne[i][0], reg_a_tsne[i][1], color='b', marker='.', s=4)
plt.savefig(data_dir+'tsne_reg_labelled_v.png')

plt.figure()
# ax = fig.add_subplot(111, projection='3d')

for i in range(len(reg_v_tsne)):
	if(reg_a_labels[i]>0):
		plt.scatter(reg_v_tsne[i][0], reg_v_tsne[i][1], color='r', marker='o', s=4)
	else:
		plt.scatter(reg_v_tsne[i][0], reg_v_tsne[i][1], color='b', marker='o', s=4)
 
for i in range(len(reg_a_tsne)):
	if(reg_a_labels[i]>0):
			plt.scatter(reg_a_tsne[i][0], reg_a_tsne[i][1], color='r', marker='.', s=4)
	else:
			plt.scatter(reg_a_tsne[i][1], reg_a_tsne[i][1], color='b', marker='.', s=4)
plt.savefig(data_dir+'tsne_reg_labelled_a.png')

### **Latent Image-Chunk Comparison**

In [0]:
encoder = Encoder()
encoder.apply(weights_init)
encoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/AFEW_data/encoder'))

decoder = Decoder()
decoder.apply(weights_init)
decoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/AFEW_data/decoder'))

reg_encoder = AFEW_Reg_Encoder()
reg_encoder.apply(weights_init)
reg_encoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/AFEW_data/reg_encoder'))

reg_decoder = AFEW_Reg_Decoder()
reg_decoder.apply(weights_init)
reg_decoder.load_state_dict(torch.load('/content/drive/My Drive/AFC_Project_data/AFEW_data/reg_decoder'))

batch_size = 64

print('Loading Dataset...')
data_dir = '/content/drive/My Drive/AFC_Project_data/AFEW_data/processed_images/'
transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])
dset = ImageFolderWithPathsAFEW(data_dir, transform=transform)
print(dset.imgs[:10])
data_object = torch.utils.data.DataLoader(dset, batch_size=batch_size, shuffle=True, drop_last=True)

X1 = torch.zeros(batch_size, 3, 64, 64)

if torch.cuda.is_available():
  encoder.cuda()
  decoder.cuda()
  reg_encoder.cuda()
  reg_decoder.cuda()  
  X1 = X1.cuda()

ground_truths_v = []
ground_truths_a = []
regs_v = []
regs_a = []
vanillas_v = []
vanillas_a = []

with torch.no_grad():
  for iteration, (data_, _, labels) in tqdm(enumerate(data_object)):
      image_batch_1 = data_
      X1.copy_(image_batch_1)

      print(X1.shape)

      vanilla_z_v, _, _, vanilla_z_a, _, _, _, _, _ = encoder(X1)
      reg_z_v, _, _, reg_z_a, _, _, _, _, _, _, _ = reg_encoder(X1)

      vanilla_v_tsne = TSNE(n_components=2, perplexity=100).fit_transform(vanilla_z_v.detach().cpu().numpy())
      vanilla_a_tsne = TSNE(n_components=2, perplexity=100).fit_transform(vanilla_z_a.detach().cpu().numpy())

      reg_v_tsne = TSNE(n_components=2, perplexity=100).fit_transform(reg_z_v.detach().cpu().numpy())
      reg_a_tsne = TSNE(n_components=2, perplexity=100).fit_transform(reg_z_a.detach().cpu().numpy())

      ground_truths_v = ground_truths_v + list(labels[0].detach().cpu().numpy())
      ground_truths_a = ground_truths_a + list(labels[1].detach().cpu().numpy())

      regs_v = regs_v + list(np.mean(reg_v_tsne, axis=1)/25)
      regs_a = regs_a + list(np.mean(reg_a_tsne, axis=1)/25)

      vanillas_v = vanillas_v + list(np.mean(vanilla_v_tsne, axis=1))
      vanillas_a = vanillas_a + list(np.mean(vanilla_a_tsne, axis=1))

      if(iteration==0):
          break

data_dir = '/content/drive/My Drive/AFC_Project_data/AFEW_data/'

plt.figure()
# ax = fig.add_subplot(111, projection='3d')

plt.scatter(ground_truths_v[0], ground_truths_a[0], c='red', marker='o', s=12, label='GT')
for i in range(1, batch_size):
	plt.scatter(ground_truths_v[i], ground_truths_a[i], c='red', marker='o', s=12)

plt.scatter(vanillas_v[0], vanillas_a[0], c='blue', marker='o', s=12, label='Vanilla VAE') 
for i in range(1, batch_size):
	plt.scatter(vanillas_v[i], vanillas_a[i], c='blue', marker='o', s=12)
 
plt.scatter(regs_v[0], regs_a[0], c='green', marker='o', s=12, label='Reg VAE')
for i in range(1, batch_size):
	plt.scatter(regs_v[i], regs_a[i], c='green', marker='o', s=12)

plt.legend(loc='best') 
plt.savefig(data_dir+'tsne_latent_comparison.png')
plt.close()

Loading Dataset...
[('/content/drive/My Drive/AFC_Project_data/AFEW_data/processed_images/001/00000.png', 0), ('/content/drive/My Drive/AFC_Project_data/AFEW_data/processed_images/001/00001.png', 0), ('/content/drive/My Drive/AFC_Project_data/AFEW_data/processed_images/001/00002.png', 0), ('/content/drive/My Drive/AFC_Project_data/AFEW_data/processed_images/001/00003.png', 0), ('/content/drive/My Drive/AFC_Project_data/AFEW_data/processed_images/001/00004.png', 0), ('/content/drive/My Drive/AFC_Project_data/AFEW_data/processed_images/001/00005.png', 0), ('/content/drive/My Drive/AFC_Project_data/AFEW_data/processed_images/001/00006.png', 0), ('/content/drive/My Drive/AFC_Project_data/AFEW_data/processed_images/001/00007.png', 0), ('/content/drive/My Drive/AFC_Project_data/AFEW_data/processed_images/001/00008.png', 0), ('/content/drive/My Drive/AFC_Project_data/AFEW_data/processed_images/001/00009.png', 0)]


torch.Size([64, 3, 64, 64])
